### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__COVID-19 World Vaccination Progress__  
URL:  
・https://www.kaggle.com/gpreda/covid-world-vaccination-progress    
DATA:  
・country_vaccinations.csv  
・country_vaccinations_by_manufacturer.csv

---

__Annotated geo-json geometry files for the world__  
URL:  
・https://github.com/johan/world.geo.json  
DATA:  
・countries.geo.json

In [ ]:
%matplotlib inline

import folium
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

### 1. データの概要を把握
まずデータの概要を掴んでみましょう。

国別ワクチン接種状況データをロード。

In [ ]:
country_vaccinations_df = pd.read_csv('data/country_vaccinations.csv')

len(country_vaccinations_df)

データの中身を確認。  
不明な属性があれば、データ取得元で確認します。

In [ ]:
country_vaccinations_df.head()

`info()`で列の情報を取得します。

以下はどんな属性でしょうか？  
- `people_fully_vaccinated`
- `daily_vaccinations_raw`
- `daily_vaccinations`
- `people_vaccinated_per_hundred`

In [ ]:
country_vaccinations_df.info()

`source_name`、`source_website`列は今回必要ないので削除します。

In [ ]:
country_vaccinations_df.drop(columns={'source_name', 'source_website'}, inplace=True)

country_vaccinations_df.info()

数値属性の集計情報を確認。

どんなことが読み取れますか？

In [ ]:
country_vaccinations_df.describe()

データフレームの`hist()`を使ってヒストグラムを表示します。

どんなことが読み取れますか？  
※「1e9」は10の9乗の意味です。

In [ ]:
country_vaccinations_df.hist(bins=20, figsize=(20, 12)); # セミコロンでテキスト出力を抑える

### 2. 地図を使ってデータを把握

ここでは、各国をワクチン２回接種の割合で色分けしてみます。

各国ごとの最新のワクチン２回接種割合のデータフレームを作成。

In [ ]:
country_vaccinations_df.groupby(['country', 'iso_code'])['people_fully_vaccinated_per_hundred'] \
                               .max()

In [ ]:
country_fully_vaccinated_df = country_vaccinations_df \
                               .groupby(['country', 'iso_code'])['people_fully_vaccinated_per_hundred'] \
                               .max() \
                               .reset_index() \
                               .sort_values('people_fully_vaccinated_per_hundred', ascending=False)

country_fully_vaccinated_df.head(20)

棒グラフで分布を確認。

`people_fully_vaccinated_per_hundred`の値が100%を超えているのはなぜでしょうか？

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))

sns.barplot(
    x=country_fully_vaccinated_df['country'],
    y=country_fully_vaccinated_df['people_fully_vaccinated_per_hundred'])

# x軸目盛りとラベルを表示しない
ax.set(xticklabels=[])
ax.tick_params(bottom=False)

`folium.Choropleth()`でコロプレスマップを描画します。  
国ごとの区画情報は`countries.geo.json`を利用します。

どんなことが読み取れますか？

In [ ]:
m = folium.Map(location=[0,0], zoom_start=2.4)

folium.Choropleth(
    geo_data='data/countries.geo.json',
    data=country_fully_vaccinated_df,
    fill_opacity=1,
    line_color='black',
    nan_fill_color='#888888',
    columns=['iso_code', 'people_fully_vaccinated_per_hundred'],
    key_on= "feature.id",
    fill_color='RdBu').add_to(m)

m

### 3. 少しだけ掘り下げた分析をしてみる

各製造メーカー製ワクチンの接種状況がどのように推移しているかを分析してみます。

製造メーカー別のワクチン接種データをロード。

In [ ]:
country_vaccinations_by_manufacturer_df = pd.read_csv('data/country_vaccinations_by_manufacturer.csv')

len(country_vaccinations_by_manufacturer_df)

データの中身を確認。

In [ ]:
country_vaccinations_by_manufacturer_df.head(10)

`info()`で各列の情報を確認。

不明な列はありますか？

また、欠損値はありますか？

In [ ]:
country_vaccinations_by_manufacturer_df.info()

グラフのX軸目盛用に`month`列を追加。

In [ ]:
country_vaccinations_by_manufacturer_df['month'] = pd.to_datetime(
    country_vaccinations_by_manufacturer_df.date, format='%Y-%m-%d').dt.strftime('%Y-%m')

country_vaccinations_by_manufacturer_df['month']

グラフを表示した時に各ワクチンが同じ色でされるよう、`vaccine`列をカテゴリ型に変換します。

In [ ]:
country_vaccinations_by_manufacturer_df['vaccine'] = country_vaccinations_by_manufacturer_df['vaccine'] \
                                                     .astype('category')

country_vaccinations_by_manufacturer_df['vaccine']

国、ワクチン、月別でグループ化し、接種回数を計算したデータフレームを生成。

`total_vaccinations`が累計という前提で、最大値を取得します。

In [ ]:
location_vaccine_month_df = country_vaccinations_by_manufacturer_df \
                             .groupby(['location', 'vaccine', 'month']) \
                             .agg({'total_vaccinations': 'max'})

location_vaccine_month_df.head(20)

`vaccine`列をカテゴリ型にしたため、`location`、`vaccine`、`month`のすべての組み合わせの行が作られ、存在しない組み合わせの場合は`total_vaccinations`列の値が`NaN`になってしまいました。

日本の`Pfizer/BioNTech`接種回数にも`NaN`が含まれています。

In [ ]:
location_vaccine_month_df.query('location == "Japan" and vaccine == "Pfizer/BioNTech"')

これを国・ワクチンのグループ単位で`fillna()`を使って修正します。

先頭から`NaN`の場合は`fillna(0)`で`0`を埋め、後続の値が`NaN`の場合は`fillna(method='ffill')`で同じ値で埋めます。

In [ ]:
location_vaccine_month_df = location_vaccine_month_df \
                             .groupby(['location', 'vaccine']) \
                             .apply(lambda g: g.fillna(method='ffill').fillna(0))

location_vaccine_month_df.head(20)

もう一度、日本の`Pfizer/BioNTech`接種状況を確認します。

In [ ]:
location_vaccine_month_df.query('location == "Japan" and vaccine == "Pfizer/BioNTech"')

すべての行で`NaN`がないことも確認します。

In [ ]:
print(location_vaccine_month_df['total_vaccinations'].isna().sum())

問題ないようであれば、今度は世界全体のデータを計算します。

ワクチン、月別でグループ化し、接種回数を計算したデータフレームを生成し、月でソートします。

今度は`total_vaccinations`の合計値を取得します。

In [ ]:
world_vaccine_month_df = location_vaccine_month_df \
                          .groupby(['vaccine', 'month']) \
                          .agg({'total_vaccinations': 'sum'}) \
                          .sort_values('month')

world_vaccine_month_df.head()

折れ線グラフを表示します。

どんなことが読み取れますか？

In [ ]:
sns.set(palette='Set2', style='white', font=['MS Gothic','Hiragino Sans','TakaoPGothic']) # 日本語フォントは設定が必要

fig, ax = plt.subplots(figsize=(20,12))

sns.lineplot(data=world_vaccine_month_df,
             x='month',
             y='total_vaccinations',
             hue='vaccine')

ax.set_xlabel("月");
plt.xticks(rotation=90)

ax.set_ylabel("ワクチン総接種回数（億）");
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: '{:,.0f}'.format(y/100_000_000))) # 億単位

ax.legend(loc="lower left", bbox_to_anchor=(1.01, 0.8))

同様に、任意の8ヵ国についてワクチン別の接種回数の推移をグラフで表示してみましょう。

任意の8ヵ国のリストを取得します。

In [ ]:
# 国の数は任意の数を指定可能
random_countries = np.random.choice(location_vaccine_month_df.index.get_level_values('location').unique(),
                                    size=8,
                                    replace=False)

random_countries.sort()

random_countries

８つの折れ線グラフをまとめて表示します。

どんなことが読み取れますか？

In [ ]:
nrows = math.ceil(len(random_countries) / 2) # pythonのroundは「偶数丸め」
ncols = 2

fig, axes = plt.subplots(nrows, ncols, figsize=(20, 7*nrows), sharex=True)

for i in range(nrows):
    for j in range(ncols):
    
        target_ax = axes[i, j] if nrows > 1 else axes[j]
        
        # 最後の行のグラフにあるX軸ラベルを縦にする
        if i == nrows-1:
            target_ax.tick_params(labelrotation=90)
    
        country_index = i*ncols + j
        if country_index == len(random_countries):
            break

        target_country = random_countries[country_index]
        sns.lineplot(data=location_vaccine_month_df.query('location == @target_country').sort_values('month'),
                     x='month',
                     y='total_vaccinations',
                     hue='vaccine',
                     ax=target_ax)
        
        target_ax.set_title(target_country)
        
        target_ax.set_xlabel("月");
        
        target_ax.set_ylabel("ワクチン総接種回数（万）");
        target_ax.yaxis.set_major_formatter(
            ticker.FuncFormatter(lambda y, pos: '{:,.0f}'.format(y/10_000))); # 万単位
    
        target_ax.legend(loc="upper left")

最後に、先ほどの任意の8ヵ国での`Pfizer/BioNTech`製ワクチンの接種回数の推移を１つの折れ線グラフで表示してみましょう。

まずグラフ用のデータフレームを用意します。

条件による絞り込みは`query()`を使うと簡単かもしれません。

In [ ]:
### チャレンジしてみましょう！！ ###

グラフを表示します。

どんなことが読み取れますか？

In [ ]:
### チャレンジしてみましょう！！ ###